In [9]:
import requests
import xmltodict
import datetime
from dateutil import tz
import pytz
import xmltodict
from astropy.time import Time
import ephem
import PES_secrets

In [10]:
entry = requests.get("https://targettool.aavso.org/TargetTool/api/v1/telescope",auth=(PES_secrets.apikey,"api_token"))
print(entry.json())


{'latitude': -35.351835, 'longitude': 149.030122, 'targetaltitude': 20.0, 'sunaltitude': -5.0}


In [11]:
# lets now consider the night in question
# Make an observer
wombat = ephem.Observer()

# Set the date and time 
wombat.date = datetime.datetime.today().strftime('%Y-%m-%d')#"2024-07-14"

# Location 
wombat.lon = str(148.12899)
wombat.lat = str(-35.282)

# Elevation 
wombat.elev = 500

# To get U.S. Naval Astronomical Almanac values, use these settings
wombat.pressure = 0
wombat.horizon = '-0:34'

# Calculate sunrise, solar noon, and sunset
sunrise = wombat.next_rising(ephem.Sun(), use_center=True)
#noon = wombat.next_transit(ephem.Sun(), start=sunrise)
sunset = wombat.next_setting(ephem.Sun(), use_center=True)

# Relocate the horizon to get twilight times
wombat.horizon = '-12'  # -6=civil twilight, -12=nautical, -18=astronomical
beg_twilight = wombat.next_rising(ephem.Sun(), use_center=True)
end_twilight = wombat.next_setting(ephem.Sun(), use_center=True)

# Print the results
#print(f"Noon: {noon}")
print(f"Sunset: {sunset}")
#print(f"End nautical twilight: {end_twilight}")
#print(f"Begin nautical twilight: {beg_twilight}")
print(f"Sunrise: {sunrise}")
#print(f"date start: {(sunrise):.6f}")
#print(f"date end: {(sunset):.6f}")

Sunset: 2024/7/28 07:21:07
Sunrise: 2024/7/28 21:06:26


In [12]:
obs_sections = ['eb'] #ac,ep,cv,eb,spp,lpv,yso,het,misc,all.

In [13]:
entry = requests.get("https://targettool.aavso.org/TargetTool/api/v1/targets",\
                     auth=(PES_secrets.apikey,"api_token"),params={'observable':['True'],'obs_section':[obs_sections]})
targets = entry.json()

In [14]:
targets['targets']
for star_info in targets['targets']:
    star_name = star_info.get('star_name')
    if star_name:
        # for each star name look up vsx to find the ephemeris
        fmt_star_name = star_name.replace(" ","+")
        vsx_req = requests.get(f"https://www.aavso.org/vsx/index.php?view=query.votable&ident={fmt_star_name}&format=xml")
        json_data = xmltodict.parse(vsx_req.content)
        try:
            const = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][2]
            radec = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][3]
            vartyp = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][4]
            vmin = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][5]
            vmax = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][7]
            photsys = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][6]
            epoch = float(json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][9])
            period = float(json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][11])
        except:
            print(f'error reading {star_name}')
            continue
        #print(f"{const}, {radec}, {vartyp}, {vmin}, {vmax}, {epoch}, {period}")
        # skip those without an ephemeris
        if (epoch==0 or period==0):
            #print(f"{epoch} {period}")
            continue
        # for each star we check the time of minimum at this date
        # to do so we take the epoch zero and add multiples of the period
        # until the date is near the night of observation
        mjd_start = (end_twilight+2415020.0-2400000.5)
        mjd_end = (beg_twilight+2415020.0-2400000.5)
        ep = epoch
        while (ep<mjd_start):
            ep += period
        ep -= 0.5 # somewhere there is a jd and not a mjd
        #print(f"Minimum at: {ep}")
        if (ep > mjd_start and ep < mjd_end):
            print(f"\nTarget Name: {star_name} {radec}")
            #print(f"well placed for observing {wombat.date}")
            jd_value = ep + 2400000.5 ### this is correct

            # Create an astropy Time object from JD
            t = Time(jd_value, format='jd')
            # Convert to datetime (UTC)
            utc_datetime = t.to_datetime(tz.UTC)
            print(f"UTC Date and Time: {utc_datetime}")
            target_timezone = pytz.timezone('Australia/Canberra')
            local_datetime = utc_datetime.astimezone(target_timezone)
            print(f"LT Mimimum: {local_datetime.isoformat()}")
            (ra, dec) = [(float(x)/180.)*3.141592 for x in radec.split(',')]
            object = ephem.FixedBody()
            object._ra = ra
            object._dec = dec
            object._epoch = 2000.0
            wombat.horizon = '30' # set this to determine z=2 limit
            object.compute(wombat)
            try:
                rise = wombat.next_rising(object) 
                set = wombat.next_setting(object)
            except:
                print("Target outside engagement zone")
                continue
            # now we have the bounding sunset and sunrise + the object rise and set
            # we can determine the time that the object is visible
            start_obs = max(rise, end_twilight)
            end_obs = min(set, beg_twilight)
            print(f"Twi: {end_twilight} to {beg_twilight}")
            print(f"Target: {rise} to {set}")
            print(f"Limits: {start_obs} to {end_obs}")
            if (rise > beg_twilight):
                print("Target enters zone too late")
                continue
            if (ep < end_obs and utc_datetime > ep):
                print("***Target has minimum in the targeting window***")
            print(f"Duration: {(end_obs - start_obs) * 24:.2f} hrs where target z>2 and nautical")


error reading ASAS J050610+2638.7

Target Name: DK Cir 221.80270000,-57.67730000
UTC Date and Time: 2024-07-28 12:30:14.400096+00:00
LT Mimimum: 2024-07-28T22:30:14.400096+10:00
Twi: 2024/7/28 08:20:21 to 2024/7/28 20:07:18
Target: 2024/7/28 02:43:33 to 2024/7/28 14:31:44
Limits: 2024/7/28 08:20:21 to 2024/7/28 14:31:44
Duration: 6.19 hrs where target z>2 and nautical

Target Name: EW Cru 182.64917000,-63.25064000
UTC Date and Time: 2024-07-28 12:06:45.217315+00:00
LT Mimimum: 2024-07-28T22:06:45.217315+10:00
Twi: 2024/7/28 08:20:21 to 2024/7/28 20:07:18
Target: 2024/7/28 23:43:57 to 2024/7/28 12:09:51
Limits: 2024/7/28 23:43:57 to 2024/7/28 12:09:51
Target enters zone too late

Target Name: OU Lup 226.70125000,-35.08264000
UTC Date and Time: 2024-07-28 09:11:45.024299+00:00
LT Mimimum: 2024-07-28T19:11:45.024299+10:00
Twi: 2024/7/28 08:20:21 to 2024/7/28 20:07:18
Target: 2024/7/28 03:53:49 to 2024/7/28 13:57:36
Limits: 2024/7/28 08:20:21 to 2024/7/28 13:57:36
Duration: 5.62 hrs where 

<h1>Check Target</h1>

Next section to develop a check of if the target is up for long enough

In [ ]:
radec = '180, -30'

In [ ]:
(ra, dec) = [(float(x)/180.)*3.141592 for x in radec.split(',')]

In [ ]:
object = ephem.FixedBody()
object._ra = ra
object._dec = dec
object._epoch = 2000.0
wombat.horizon = '30'

In [ ]:
object.compute(wombat)

In [ ]:
rise = wombat.next_rising(object) #, start=sunset)
set = wombat.next_setting(object) #, start=sunset)
print(f"{rise} {set}")

In [ ]:
wombat.horizon = '-12'
local_sunrise = wombat.next_rising(ephem.Sun()) 
local_sunset = wombat.next_setting(ephem.Sun())
print(f"{local_sunset} {local_sunrise}")

In [ ]:
# now we have the bounding sunset and sunrise + the object rise and set
# we can determine the time that the object is visible
start_obs = max(rise, local_sunset)
end_obs = min(set, local_sunrise)
print(f"{start_obs} to {end_obs}")
print(f"Duration: {(end_obs - start_obs) * 24:.2f} hrs where target z>3 and nautical")

In [ ]:
mjd_rise = ephem.julian_date(rise)-2400000.5
mjd_set = ephem.julian_date(set)-2400000.5
if (mjd_set < mjd_rise):
    mjd_set += 1
print(f"{mjd_rise} {mjd_set}")

In [ ]:
mjd_set - mjd_rise

In [ ]:
(34+85)/2


In [ ]:
dec

In [ ]:
a = 180
print((a/360)*24)


In [ ]:
object.az